In [1]:
from gurobipy import *
import numpy as np

tests_per_week = 2
number_of_people = 20

availability_matrix = np.random.randint(0,2, size=(number_of_people, 7*number_of_people)) # random availabilities
# availability_matrix = np.ones((number_of_people, 7*number_of_people)) # pretend everyone is available, always

# Zero out irrelevant parts of the matrix
for i in range(len(availability_matrix)):
    if i > 0:
        for j in range(i*7):
            availability_matrix[i][j] = 0
    if i < len(availability_matrix):
        for j in range((i+1)*7, 7*number_of_people):
            availability_matrix[i][j] = 0
            
# Dealing with infeasibles...
people_total_availability = np.sum(availability_matrix, 1)
infeasible_people = np.where(people_total_availability < tests_per_week)

# Enforce that everyone gets two tests a week
Aeq = availability_matrix
beq = np.multiply(tests_per_week, np.ones((number_of_people,1)))

# Loosen beq in the case where people aren't available enough
for i in range(len(infeasible_people[0])):
    beq[infeasible_people[0][i]][0] = people_total_availability[infeasible_people[0][i]]

# This picks up the number of total tests (per?) person each day of the week
testing_capacity_matrix = np.zeros((7, 7*number_of_people))
for i in range(len(testing_capacity_matrix)):
    for j in range(i, 7*number_of_people, 7):
        testing_capacity_matrix[i][j] = 1
        
A = testing_capacity_matrix
b = np.ceil(np.multiply(number_of_people*tests_per_week/7, np.ones((7,1)))) # why is this ceil? why o

# x is all zeros except for the 1 schedule choice per-person ??

# Gurobi optimization, still a little confused here.
m = Model()

# Variables
x0 = m.addVars(7*number_of_people, name='x0', vtype=GRB.BINARY) # x \in [0,1]

m.setObjective(x0.sum(), GRB.MAXIMIZE) # is maximize right? we want people to take as many tests as poss.

# Ax <= b
for i in range(7):
    m.addConstr(sum(x0[j]*A[i][j] for j in range(7*number_of_people)) <= b[i][0])

# Aeq x = beq
for i in range(number_of_people):
    m.addConstr(sum(x0[j]*Aeq[i][j] for j in range(7*number_of_people)) == beq[i][0])

m.optimize()

print()
for i in range(number_of_people):
    idx = 7*i
    num_tests = 0
    days_avail = 0
    for j in range(7):
        num_tests += x0[idx].x
        days_avail += Aeq[i][idx]
        idx += 1
    if (days_avail < num_tests):
        print("!!! Person %02d: available days = %d --> %d tests !!!" % (i, days_avail, num_tests))
    else:
        print("Person %02d: available days = %d --> %d tests" % (i, days_avail, num_tests))

Using license file /Users/gabe/gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 27 rows, 140 columns and 209 nonzeros
Model fingerprint: 0x2f8c0c70
Variable types: 0 continuous, 140 integer (140 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective 42.0000000
Presolve removed 7 rows and 83 columns
Presolve time: 0.00s
Presolved: 20 rows, 57 columns, 113 nonzeros
Variable types: 0 continuous, 57 integer (56 binary)

Root relaxation: cutoff, 12 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0        42.00000   42.00000  0.00%     -    0s

Explored 0 nodes (12 simplex iterations) in 0.07 seconds
Thread 